In [ ]:
import pandas as pd

For player in club sport - check if playing on a team
Increase counter for that team

After check if any team is above their counter

In [ ]:
club_data = pd.read_excel("Club Raw Data.xlsx")

im_data = pd.read_excel("IM Raw data.xlsx")

In [ ]:
club_data_columns = ["Sport", "Year", "Season", "GTID"]
im_data_columns = ["Last Name", "First Name", "Student ID", "Sport", "Team", "LeagueSeason", "LeagueYear"]
# print(club_data.columns)
# print(im_data.columns)

to_drop = []
for column in im_data.columns:
    if column not in im_data_columns:
        to_drop.append(column)

cleaned_im_data = im_data.drop(columns=to_drop)

to_drop = []
for column in club_data.columns:
    if column not in club_data_columns:
        to_drop.append(column)


cleaned_club_data = club_data.drop(columns = to_drop)
print(cleaned_im_data.columns)
print(cleaned_club_data.columns)

In [ ]:
# print(cleaned_club_data.Sport.unique())
# print(cleaned_im_data.Sport.unique())

generalized_equivalencies = ["Volleyball", "Basketball", "Soccer", "Billiards", "Football", "Ultimate Frisbee", "Baseball"] 

relevant_club_sports = []
for sport in cleaned_club_data.Sport.unique():
    for general in generalized_equivalencies:

        if general in sport:
            # print(sport)
            relevant_club_sports.append(sport)

irellevant = []
for sport in cleaned_club_data.Sport.unique():
    if sport not in relevant_club_sports:
        irellevant.append(sport)
print(irellevant)


In [ ]:
index_id_sport_included = cleaned_club_data.set_index("GTID")
club_sport_relevant_data = cleaned_club_data.set_index("Sport").drop(irellevant)

club_sport_relevant_data = club_sport_relevant_data.set_index("GTID")

# print(club_sport_relevant_data.index.unique())
# print(index_id_sport_included)
try:
    cleaned_im_data = cleaned_im_data.set_index("Student ID")
except:
    pass

print(cleaned_im_data.index.unique())

players_to_check = []
for im_participant in cleaned_im_data.index.unique():
    if im_participant in club_sport_relevant_data.index.unique():
        sport = index_id_sport_included.loc[im_participant].Sport
        players_to_check.append((im_participant, sport))

print(players_to_check)

In [ ]:
club_players_with_teams = []
for player, sport in players_to_check:
    if type(sport) is not str:
        sport = sport.values[-1]
    club_players_with_teams.append((cleaned_im_data.loc[player].drop_duplicates(), sport))

# print(len(club_players_with_teams))
print(club_players_with_teams[0])


In [ ]:
dict_of_club_players = {}
for player, sport in club_players_with_teams:
    
    generic = sport
    for generic_sport in generalized_equivalencies:
        if generic_sport in sport:
            generic = generic_sport
    ID = player.index[0]
    # print(ID)
    # print(player)
    for sport in player.Sport.values:
    
        if generic in sport:
            player_by_sport = player.set_index("Sport")
            team = player_by_sport.loc[sport].at["Team"]
            
            name = player_by_sport.loc[sport].at["First Name"] + " " + player_by_sport.loc[sport].at["Last Name"]
            
            
            if (team, sport) in dict_of_club_players.keys():

                list_to_add = dict_of_club_players.get((team, sport))
                list_to_add.append((ID, name))
                print(list_to_add)
                
                dict_of_club_players[(team, sport)] = list_to_add
                
            else:   
                dict_of_club_players[(team, sport)] = [(ID, name)]
# print(dict_of_club_players)
    

# THIS DOES NOT ACCOUNT FOR 3V3 BASKETBALL AT THE MOMENT
# ALSO OTHER OVERLAPPING IM-SPORTS SO INDOOR VS OUTDOOR SOCCER

In [ ]:
for sport, limit in zip(generalized_equivalencies,[2,2,5,0,3,3,4]):
    print(f"Sport: {sport} Limit: {limit}")

In [ ]:
violations = []
for sport, limit in zip(generalized_equivalencies,[2,2,5,0,3,3,4]):
    for team, im_sport in dict_of_club_players.keys():
        if sport in im_sport:
            listed_team = dict_of_club_players.get((team, im_sport))
    
            if (len(listed_team) > limit):
                violations.append((team, im_sport))


print(len(violations))
print(violations)

f = open("teams_in_violation.txt", "w")
for violation in violations:
    print(f"Team: {violation[0]}", file=f)
    print(f"Sport: {violation[1]}", file=f)
    print(f"Players: {dict_of_club_players.get(violation)}", file=f)
    print("", file=f)
f.close()
